In [1]:
import findspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.conf import SparkConf
from delta import configure_spark_with_delta_pip
conf = SparkConf()
conf.set("spark.driver.maxResultSize", "8g")
conf.set("spark.driver.memory", "8g") 
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

builder = SparkSession.builder.appName("dev_test_notebook_2") \
    .enableHiveSupport()\
    .config(conf=conf)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-57af681e-9ecd-489b-836b-7d58cd77193e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 463ms :: artifacts dl 70ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

23/07/12 12:12:05 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/07/12 12:12:05 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/07/12 12:12:05 WARN Client: Same path resource file:///home/jupyter/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS src_test2 (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH '/opt/spark/examples/src/main/resources/kv1.txt' INTO TABLE src_test2")

In [ ]:
spark.sql("SELECT * FROM src_test2").show()

In [ ]:
spark.sql("SELECT COUNT(*) FROM src_test2").show()

In [ ]:
sqlDF = spark.sql("SELECT key, value FROM src_test2 WHERE key < 10 ORDER BY key")

In [ ]:
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

In [ ]:
from pyspark.sql import Row

Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

In [ ]:
spark.sql("SELECT * FROM records r JOIN src_test2 s ON r.key = s.key").show()

In [4]:
spark.sql("SELECT * FROM default.cr_table_demo_delta").show()

23/07/12 12:13:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+-------------------+-------------+-----------------+--------------+-----------+--------------------+-----------------+--------------------+----------------------+---------------------+--------------------+-----------------+----------+---------------------------+--------------------+---------------+--------------+-------------------+--------------------+--------------------+
|Part_IsAssembly|        Part_Weight|Part_RealArea|Part_ExternalArea|Part_Thickness|Part_Length|Part_RectangularArea|Part_IsFillerPart|Part_MaterialDensity|Part_RectangularWeight|Part_MarkingPerimeter| Part_ExternalWeight|Part_CutPerimeter|Part_Width|ManufacturingOrder_Quantity|     Part_WorkCenter|  Part_Material|Part_Rotations|Part_CreationMethod|           Tenant_ID|            Plant_ID|
+---------------+-------------------+-------------+-----------------+--------------+-----------+--------------------+-----------------+--------------------+----------------------+---------------------+-----------

In [ ]:
spark.stop()